In [1]:
# I like to start by checking if the GPU was initiated succesfully,
# and by seeing the tensroflow and python versions being used
# by the notebook.

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import tensorflow as tf, sys
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print('\n' + '..'*20 + '\n')
print('tensorflow version: %s' % tf.__version__)
print('python version: %s' % sys.version)
print('python executable location: %s' % sys.executable)

# --------------------------------------------------------------
# --------------------------------------------------------------

%matplotlib inline
%load_ext autoreload
%autoreload 2

from helpers import *
import os, sys, tarfile, time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from matplotlib.ticker import MaxNLocator

params = {'legend.fontsize': 'x-large',
          'font.size'      : 10,
          'figure.figsize' : (17, 3),
          'axes.labelsize' : 'x-large',
          'axes.titlesize' : 'x-large',
          'axes.grid'      : 'on',
          'xtick.labelsize': 'x-large',
          'ytick.labelsize': 'x-large'}

pylab.rcParams.update(params)


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14614415894446553231
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11324823962
locality {
  bus_id: 1
}
incarnation: 12437008016174268944
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0"
]

........................................

tensorflow version: 1.0.0
python version: 3.6.1 |Continuum Analytics, Inc.| (default, May 11 2017, 13:09:58) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
python executable location: /home/aind2/anaconda3/envs/aind-dog/bin/python


In [2]:
# Read the Lab41 dataset into 
# train_Y, cv_Y, test_Y
# train_Y_indices, cv_Y_indices, test_Y_indices

start = time.time()
#...................................................

RATINGS_PATH = './data_ml-20m/ratings.csv'
MOVIES_PATH = './data_ml-20m/movies.csv'

#...................................................

with open(RATINGS_PATH) as f:
    ratings = f.readlines()

with open(MOVIES_PATH) as f:
    movies = f.readlines()

#...................................................

from_mID_2_idx = {}
for i, row in enumerate(movies[1:]):
    mID = int(row.split(',')[0])
    from_mID_2_idx[mID] = i
    
#...................................................

n_users = 138493
n_movies = len(movies[1:]) # n_movies = 27278

N = len(ratings[1:])
Y_indices = np.zeros((N,2), dtype=np.int32)
Y = np.zeros(N)

for i, row in enumerate(ratings[1:]):
    uID, mID, yij = [e for e in row.split(',')[:-1]]
    u_idx = int(uID) - 1
    v_idx = from_mID_2_idx[int(mID)]
    yij = float(yij)
    
    Y_indices[i] = [u_idx, v_idx]
    Y[i] = yij

#...................................................

i1 = int(N*.64)
i2 = int(N*.8)

train_Y = Y[:i1]
train_Y_indices = Y_indices[:i1]

cv_Y = Y[i1:i2]
cv_Y_indices = Y_indices[i1:i2]

test_Y = Y[i2:]
test_Y_indices = Y_indices[i2:]

del Y, Y_indices

print_runtime(start)

Runtime: 1 min 24 sec


In [249]:
start = time.time()
#..............................................
BATCH_SIZE = 1024*16
LAMBDA = 0.0001
k = 10 

Y = tf.placeholder(dtype=tf.float32, shape=(BATCH_SIZE,))
Y_indices = tf.placeholder(dtype=tf.int32, shape=(BATCH_SIZE,2))

# initialization of U and V is critical. 
# set mean=np.sqrt(mu/k) , where mu ~ 3 or 3.5
U = tf.Variable(tf.truncated_normal(shape=(n_users,k), mean=np.sqrt(3.5/k), stddev=0.2), dtype=tf.float32)
V = tf.Variable(tf.truncated_normal(shape=(n_movies,k), mean=np.sqrt(3.5/k), stddev=0.2), dtype=tf.float32)

sliced_U, sliced_V = get_sliced_UV(Y_indices, Y, U, V, k, BATCH_SIZE)

#..............................................     

reg = LAMBDA*(tf.reduce_sum((U)**2) + tf.reduce_sum((V)**2)) / BATCH_SIZE / k
loss = (tf.norm(Y - tf.reduce_sum(tf.multiply(sliced_U, sliced_V), axis=1))) / BATCH_SIZE + reg
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

#.............................................. 
print_runtime(start) 

Runtime: 0 min 0 sec


In [250]:
start = time.time()
#.............................................. 
NUM_EPOCHS = 20
init = tf.global_variables_initializer()
batch = Batch(train_Y_indices, train_Y, BATCH_SIZE=BATCH_SIZE)
n_batches = len(train_Y) // BATCH_SIZE
print('n_batches: {}'.format(n_batches))
batch_no = 0
mae = []

with tf.Session() as sess:
    sess.run(init)
    while not (batch.epoch == NUM_EPOCHS and batch.last_batch==True):
        batch_no += 1
        batch_Y_indices, batch_Y = batch.next()
        if not batch.broken:
            train_acc, _loss = sess.run([train, loss], 
                                      feed_dict={Y_indices : batch_Y_indices, Y : batch_Y})
            print("batch_no: {}, _loss: {} ".format(batch_no, _loss), end='\r') 
        else: 
           
            batch_no = 0
            _U, _V = sess.run([U, V], feed_dict={}) 
            preds, _mae = evaluate(cv_Y, cv_Y_indices, _U, _V, BATCH_SIZE)
            _, _mae_test = evaluate(test_Y, test_Y_indices, _U, _V, BATCH_SIZE)
            mae.append(_mae)
            mean_preds = np.mean(preds)
            print('\nmae: %6.4f, _mae_test: %6.4f, mean_preds: %6.4f' % (_mae,_mae_test,mean_preds))
            print('reg/_loss: %6.4f' %(sess.run(reg/_loss)))

#.............................................. 
print_runtime(start)

n_batches: 781

New epoch: 1 ******************************
batch_no: 781, _loss: 0.008144455030560493  
mae: 0.7930, _mae_test: 0.7889, mean_preds: 3.4806
reg/_loss: 0.0445

New epoch: 2 ******************************
batch_no: 781, _loss: 0.007703713607043028  
mae: 0.8349, _mae_test: 0.8322, mean_preds: 3.2432
reg/_loss: 0.0447

New epoch: 3 ******************************


KeyboardInterrupt: 